In [78]:
import numpy as np
import pandas as pd
import os

import matplotlib
#matplotlib.use('agg')
import matplotlib.pyplot as plt
import seaborn as sns

import plotly
plotly.tools.set_credentials_file(username='vvr', api_key='Ker155UU70wk3Ccz6iyM')
import plotly.plotly as py
import plotly.graph_objs as go

color = sns.color_palette("husl", 8)

import sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer


%matplotlib inline

import squarify

import nltk
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
#from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
import re



from scipy.sparse import hstack


In [79]:
train= pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data = pd.read_csv('campaign_data.csv')
sub = pd.read_csv('sample_submission_4fcZwvQ.csv')

In [80]:
train.columns  = ['id', 'userid', 'campaignid', 'send_date', 'open', 'click']
test.columns  = ['id', 'userid', 'campaignid', 'send_date']

In [81]:
train.columns

Index(['id', 'userid', 'campaignid', 'send_date', 'open', 'click'], dtype='object')

In [82]:
test.columns

Index(['id', 'userid', 'campaignid', 'send_date'], dtype='object')

In [83]:
data['communication_type'].value_counts()

Hackathon          13
Corporate          12
Newsletter          9
Conference          8
Upcoming Events     7
Others              2
Webinar             1
Name: communication_type, dtype: int64

In [84]:
data[['campaign_id', 'communication_type']].sort_values(['communication_type','campaign_id'])

,campaign_id,communication_type
2,31,Conference
3,32,Conference
7,36,Conference
8,37,Conference
9,38,Conference
11,40,Conference
20,49,Conference
24,53,Conference
12,41,Corporate
14,43,Corporate


In [85]:
data[['campaign_id', 'communication_type','total_links']].sort_values(['communication_type','campaign_id'])

,campaign_id,communication_type,total_links
2,31,Conference,15
3,32,Conference,24
7,36,Conference,13
8,37,Conference,9
9,38,Conference,28
11,40,Conference,10
20,49,Conference,119
24,53,Conference,104
12,41,Corporate,14
14,43,Corporate,11


In [86]:
data[['campaign_id', 'communication_type','total_links','no_of_images']].sort_values(['communication_type',
                                                                                      'campaign_id', 'total_links'])

,campaign_id,communication_type,total_links,no_of_images
2,31,Conference,15,5
3,32,Conference,24,7
7,36,Conference,13,2
8,37,Conference,9,3
9,38,Conference,28,12
11,40,Conference,10,2
20,49,Conference,119,16
24,53,Conference,104,13
12,41,Corporate,14,2
14,43,Corporate,11,3


In [87]:
import re

data['subject'] = data['subject'].map(lambda x: re.sub(r'\W+', ' ', x))
data['email_body'] = data['email_body'].map(lambda x: re.sub(r'\W+', ' ', x))

In [88]:
def pir(data):
    t = data.text.str.split(expand=True).stack()
    return t.loc[t.str.len() >= 3].groupby(level=0).apply(' '.join)
data['subject'] = data['subject'].str.findall('\w{3,}').str.join(' ')
data['email_body'] = data['email_body'].str.findall('\w{3,}').str.join(' ')

In [89]:
import nltk
data['subject'] = data.apply(lambda row: nltk.word_tokenize(row['subject']), axis=1)
data['email_body'] = data.apply(lambda row: nltk.word_tokenize(row['email_body']), axis=1)


In [90]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

data['subject'] = data["subject"].apply(lambda x: [stemmer.stem(y) for y in x])
data['email_body'] = data["email_body"].apply(lambda x: [stemmer.stem(y) for y in x])


In [91]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['subject'] = data['subject'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))
data['email_body'] = data['email_body'].apply(lambda x: ' '.join([word for word in x if word not in (stop)]))

In [92]:
train.head(3)

,id,userid,campaignid,send_date,open,click
0,42_14051,14051,42,01-09-2017 19:55,0,0
1,52_134438,134438,52,02-11-2017 12:53,0,0
2,33_181789,181789,33,24-07-2017 15:15,0,0


In [93]:
test.head()

,id,userid,campaignid,send_date
0,63_122715,63,122715,01-02-2018 22:35
1,56_76206,56,76206,02-01-2018 08:15
2,57_96189,57,96189,05-01-2018 18:25
3,56_166917,56,166917,02-01-2018 08:15
4,56_172838,56,172838,02-01-2018 08:12


In [94]:
train.shape

(1023191, 6)

In [95]:
test.shape

(773858, 4)

# TFIDF

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer(ngram_range = (6,6))
data['subject'] =  v.fit_transform(data['subject'])

In [97]:
data['email_body'] =  v.fit_transform(data['email_body'])

# Label Encoding

In [98]:
le = LabelEncoder()
le.fit(data['communication_type'])
data['communication_type'] = le.transform(data['communication_type'])
#data.head(3)

In [99]:
data.drop('email_url', axis=1, inplace = True)

# Concating

In [71]:
train.set_index('campaignid', inplace = True)

In [72]:
test.set_index('campaignid', inplace = True)

In [73]:
data.set_index('campaign_id', inplace = True)

In [74]:
train = pd.merge(train, data, left_index=True, right_index=True)

In [100]:
test = pd.merge(test, data, left_index=True, right_index=True, )

# Multi Label Classification

In [102]:
train.shape

(1023191, 6)

In [103]:
test.shape

(52, 12)